<!-- Team Banner -->
<div align="center">
  <h1>🌟<strong>EVER LEARNERS</strong>🌟</h1>
</div>


<!-- Team Banner -->
<div align="center">
  <h2><strong>KRAKEN TAXA RETRIEVAL FOR MICROBIOME DATA</strong></h2>
</div>

<div align="center">
  <img src="https://zindi-public-release.s3.eu-west-2.amazonaws.com/uploads/competition/image/591/thumb_f7f99618-bc3d-44db-acfa-39b971d7472f.png" alt="Description" style="width:50%;"/>
</div>


In [1]:
%%capture
!pip install Bio kaggle pyfiglet--quiet

In [2]:
%%capture
!apt-get update
!apt-get install -y kraken2

In [3]:
# Download minikraken (8GB - more manageable)
!wget ftp://ftp.ccb.jhu.edu/pub/data/kraken2_dbs/old/minikraken2_v2_8GB_201904.tgz
!tar -xvzf minikraken2_v2_8GB_201904.tgz

--2025-11-01 02:24:45--  ftp://ftp.ccb.jhu.edu/pub/data/kraken2_dbs/old/minikraken2_v2_8GB_201904.tgz
           => ‘minikraken2_v2_8GB_201904.tgz’
Resolving ftp.ccb.jhu.edu (ftp.ccb.jhu.edu)... 128.220.174.63
Connecting to ftp.ccb.jhu.edu (ftp.ccb.jhu.edu)|128.220.174.63|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/kraken2_dbs/old ... done.
==> SIZE minikraken2_v2_8GB_201904.tgz ... 5935990636
==> PASV ... done.    ==> RETR minikraken2_v2_8GB_201904.tgz ... done.
Length: 5935990636 (5.5G) (unauthoritative)

minikraken2_v2_8GB_ 100%[===================>]   5.53G  40.9MB/s    in 2m 25s  

2025-11-01 02:27:12 (39.1 MB/s) - ‘minikraken2_v2_8GB_201904.tgz’ saved [5935990636]

minikraken2_v2_8GB_201904_UPDATE/
minikraken2_v2_8GB_201904_UPDATE/taxo.k2d
minikraken2_v2_8GB_201904_UPDATE/opts.k2d
minikraken2_v2_8GB_201904_UPDATE/database100mers.kmer_distrib
minikraken2_v2_8GB_201904_UPDATE/database15

In [4]:
%%capture
!apt-get update && apt-get install -y libhts-dev zlib1g libzstd1 liblzma5

# DO THIS ONLY ON KAGGLE IF YOU HAVR THE PYGENIE DATASET
!cp -r /kaggle/input/pygenie/ /kaggle/working/pygenie

# CONTINUE FROM HERE
!pip install /kaggle/working/pygenie --quiet
!pip install Bio pyfiglet --quiet

In [5]:
import pygenie
from pygenie import *

stdout, stderr, code = genie_help()
print(stdout)

[INFO,      0.000s, App]:    ______           _
[INFO,      0.000s, App]:   / ____/__  ____  (_)__
[INFO,      0.000s, App]:  / / __/ _ \/ __ \/ / _ \
[INFO,      0.000s, App]: / /_/ /  __/ / / / /  __/
[INFO,      0.000s, App]: \____/\___/_/ /_/_/\___/
[INFO,      0.000s, App]: Command: /usr/local/lib/python3.11/dist-packages/pygenie/genie help 
[ERROR,      0.000s, App]: Usage: 
[ERROR,      0.000s, App]: genie <operation> <operation specific options> 
[ERROR,      0.000s, App]: 
[ERROR,      0.000s, App]: List of operations:
[ERROR,      0.000s, App]: help
[ERROR,      0.000s, App]: run
[ERROR,      0.000s, App]: transcode-fastq
[ERROR,      0.000s, App]: transcode-sam
[ERROR,      0.000s, App]: 
[ERROR,      0.000s, App]: To learn more about an operation, type "genie <operation> --help".



In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import os
import pandas as pd
from pygenie import run
from pathlib import Path
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import warnings
warnings.filterwarnings('ignore')

# Directory containing MGB files
mgb_dir = '/kaggle/input/mpeg-g-dialogue/TrainFiles/TrainFiles'
kraken_db = 'minikraken2_v2_8GB_201904_UPDATE'

# Get all .mgb files
mgb_files = sorted([f for f in os.listdir(mgb_dir) if f.endswith('.mgb')])
print(f"Found {len(mgb_files)} MGB files to process")

# Determine number of cores to use (leave 1 core free)
n_cores = max(1, cpu_count() - 1)
print(f"Using {n_cores} CPU cores for parallel processing")

def process_single_file(mgb_file):
    """Process a single MGB file through the Kraken2 pipeline"""
    file_id = Path(mgb_file).stem
    
    # Define paths with unique names to avoid conflicts
    mgb_path = os.path.join(mgb_dir, mgb_file)
    fastq_path = f'/tmp/{file_id}.fastq'
    kraken_output = f'/tmp/kraken_output_{file_id}.txt'
    kraken_report = f'/tmp/kraken_report_{file_id}.txt'
    
    try:
        # Step 1: Convert MGB to FASTQ
        run(input_file=mgb_path, output_file=fastq_path, force=True)
        
        # Step 2: Run Kraken2 (suppress output)
        os.system(f'kraken2 --db {kraken_db} --output {kraken_output} --report {kraken_report} {fastq_path} > /dev/null 2>&1')
        
        # Step 3: Parse Kraken report
        report = pd.read_csv(kraken_report, sep='\t', header=None,
                           names=['pct', 'reads_clade', 'reads_taxon', 'rank', 'taxid', 'name'])
        
        # Extract all taxa where reads_clade > 0
        filtered = report[report['reads_clade'] > 0][['name', 'reads_clade', 'reads_taxon', 'rank']].copy()
        filtered['filename'] = file_id
        filtered.columns = ['taxa', 'reads_clade', 'reads_taxon', 'rank', 'filename']
        
        # Step 4: Clean up files
        for f in [fastq_path, kraken_output, kraken_report]:
            if os.path.exists(f):
                os.remove(f)
        
        return filtered
        
    except Exception as e:
        # Clean up on error
        for f in [fastq_path, kraken_output, kraken_report]:
            if os.path.exists(f):
                try:
                    os.remove(f)
                except:
                    pass
        return None

# Process files in parallel with progress bar
print("\nProcessing files in parallel...")
with Pool(processes=n_cores) as pool:
    results = list(tqdm(
        pool.imap(process_single_file, mgb_files),
        total=len(mgb_files),
        desc="Processing",
        unit="file"
    ))

# Filter out None results (failed files)
all_results = [r for r in results if r is not None]

if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    
    # Reorder columns
    final_df = final_df[['filename', 'taxa', 'reads_clade', 'reads_taxon', 'rank']]
    
    # Save to CSV
    output_file = 'kraken_results_all_files.csv'
    final_df.to_csv(output_file, index=False)
    
    print(f"\n{'='*60}")
    print(f"Processing complete!")
    print(f"Total files processed successfully: {len(all_results)}/{len(mgb_files)}")
    print(f"Total taxa entries: {len(final_df)}")
    print(f"Results saved to: {output_file}")
    print(f"{'='*60}")
    
    # Show preview
    print("\nPreview of results:")
    print(final_df.head(20))
    
    # Show summary by file
    print("\nSummary by file:")
    summary = final_df.groupby('filename')['reads_clade'].agg(['count', 'sum'])
    summary.columns = ['num_taxa', 'total_reads']
    print(summary.head(20))
    print(f"\n... (showing first 20 files)")
else:
    print("\nNo files were successfully processed.")

Found 2901 MGB files to process
Using 3 CPU cores for parallel processing

Processing files in parallel...


Processing: 100%|██████████| 2901/2901 [5:40:04<00:00,  7.03s/file]



Processing complete!
Total files processed successfully: 2901/2901
Total taxa entries: 954441
Results saved to: kraken_results_all_files.csv

Preview of results:
     filename                                               taxa  reads_clade  \
0   ID_AAFNOT                                       unclassified         1318   
1   ID_AAFNOT                                               root        41522   
2   ID_AAFNOT                                 cellular organisms        41522   
3   ID_AAFNOT                                           Bacteria        41522   
4   ID_AAFNOT                                Terrabacteria group        38198   
5   ID_AAFNOT                                     Actinobacteria        30222   
6   ID_AAFNOT                                     Actinobacteria        30202   
7   ID_AAFNOT                                Propionibacteriales        15749   
8   ID_AAFNOT                               Propionibacteriaceae        15720   
9   ID_AAFNOT              

In [1]:
import pyfiglet
from rich.console import Console

console = Console()
# Banner
banner = pyfiglet.figlet_format("EVER LEARNERS", font="slant")
console.print(banner, style="bold cyan")

    _______    ____________ 
   / ____/ |  / / ____/ __ \
  / __/  | | / / __/ / /_/ /
 / /___  | |/ / /___/ _, _/ 
/_____/  |___/_____/_/ |_|  
                            
    __    _________    ____  _   ____________  _____
   / /   / ____/   |  / __ \/ | / / ____/ __ \/ ___/
  / /   / __/ / /| | / /_/ /  |/ / __/ / /_/ /\__ \ 
 / /___/ /___/ ___ |/ _, _/ /|  / /___/ _, _/___/ / 
/_____/_____/_/  |_/_/ |_/_/ |_/_____/_/ |_|/____/  
                                                    